In [1]:
import sys
import os
sys.path.append("/home/ehajizad/ss_learning/deepmask-pytorch")
import argparse
import models
import numpy as np
import time
import cv2
from PIL import Image
import torch
from tools.InferDeepMask import Infer
from utils.load_helper import load_pretrain
from matplotlib import pyplot as plt
# %matplotlib widget

In [2]:
model_names = sorted(name for name in models.__dict__
                     if not name.startswith("__") and callable(models.__dict__[name]))

parser = argparse.ArgumentParser(description='PyTorch DeepMask/SharpMask evaluation')
parser.add_argument('--arch', '-a', metavar='ARCH', default='DeepMask', choices=model_names,
                    help='model architecture: ' + ' | '.join(model_names) + ' (default: DeepMask)')
parser.add_argument('--resume', default='exps/deepmask/train/model_best.pth.tar',
                    type=str, metavar='PATH', help='path to checkpoint')
parser.add_argument('--img', default='data/testImage.jpg',
                    help='path/to/test/image')
parser.add_argument('--nps', default=1, type=int,
                    help='number of proposals to save in test')
parser.add_argument('--si', default=-2.5, type=float, help='initial scale')
parser.add_argument('--sf', default=.5, type=float, help='final scale')
parser.add_argument('--ss', default=.5, type=float, help='scale step')
parser.add_argument('--img_folder', default=None,
                    help='data/image_folder')
parser.add_argument('--save', default=None,
                    help='path to save masked objects')


def range_end(start, stop, step=1):
    return np.arange(start, stop+step, step)


def main():
    
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Setup Model
    from collections import namedtuple
    Config = namedtuple('Config', ['iSz', 'oSz', 'gSz', 'batch'])
    config = Config(iSz=160, oSz=56, gSz=112, batch=1)  # default for training

    model = (models.__dict__[args.arch](config))
    model = load_pretrain(model, args.resume)
    model = model.eval().to(device)

    scales = [2**i for i in range_end(args.si, args.sf, args.ss)]
    meanstd = {'mean':[0.485, 0.456, 0.406], 'std':[0.229, 0.224, 0.225]}
    infer = Infer(nps=args.nps, scales=scales, meanstd=meanstd, model=model, device=device, timer=False)
    
    root_dir_path = args.img_folder
    save_dir_path = args.save
    
    ses_dirs = os.listdir(root_dir_path)
    
    ses_ind = 0
    
    for ses_dir in ses_dirs[8:]:
        ses_ind += 1
        obj_ind = 0
        obj_dirs = os.listdir(os.path.join(root_dir_path, ses_dir))
        
        
        for obj_dir in obj_dirs:
            obj_ind += 1
            file_names = os.listdir(os.path.join(root_dir_path, ses_dir, obj_dir))
    
            for file_name in file_names:

                # print('| start'); tic = time.time()
                im_path = os.path.join(root_dir_path, ses_dir, obj_dir, file_name)
                im = np.array(Image.open(im_path).convert('RGB'), dtype=np.float32)
                h, w = im.shape[:2]
                img = np.expand_dims(np.transpose(im, (2, 0, 1)), axis=0).astype(np.float32)
                img = torch.from_numpy(img / 255.).to(device)
                infer.forward(img)
                masks, scores = infer.getTopProps(.2, h, w)
                # toc = time.time() - tic
                # print('| done in %05.3f s' % toc)

                # for i in range(masks.shape[2]):
                res = im[:,:,::-1].copy().astype(np.uint8)
                res = res*np.expand_dims(masks[:, :, 0], axis=2)
                cv2.imwrite(os.path.join(save_dir_path, ses_dir, obj_dir, file_name), res)
            
            
            print("Session ind: ", ses_ind, "Obj ind: ", obj_ind )
                # res[:, :, 2] = masks[:, :, i] * 255 + (1 - masks[:, :, i]) * res[:, :, 2]

                # mask = masks[:, :, i].astype(np.uint8)
                # contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                # cnt_area = [cv2.contourArea(cnt) for cnt in contours]
                # cnt_max_id = np.argmax(cnt_area)
                # contour = contours[cnt_max_id]
                # polygons = contour.reshape(-1, 2)

                # predict_box = cv2.boundingRect(polygons)
                # predict_rbox = cv2.minAreaRect(polygons)
                # rbox = cv2.boxPoints(predict_rbox)
                # print('Segment Proposal Score: {:.3f}'.format(scores[i]))

                # res = cv2.rectangle(res, (predict_box[0], predict_box[1]),
                #               (predict_box[0]+predict_box[2], predict_box[1]+predict_box[3]), (0, 255, 0), 3)
                # res = cv2.polylines(res, [np.int0(rbox)], True, (0, 255, 255), 3)

                # res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
                
                # plt.imshow(res)
                # plt.show()
                # cv2.imshow('Proposal', res)
                # cv2.waitKey(0)


if __name__ == '__main__':
    global args
    sys.argv = ['', '--arch', 'DeepMask',
                '--resume', '/home/ehajizad/ss_learning/deepmask-pytorch/pretrained/deepmask/DeepMask.pth.tar',
                '--img', '/home/ehajizad/.avalanche/data/core50/core50_128x128/s7/o1/C_07_01_009.png',
                '--img_folder', '/home/ehajizad/.avalanche/data/core50/core50_128x128_original/',
                '--save','/home/ehajizad/.avalanche/data/core50/core50_128x128/']
    args = parser.parse_args()
    main()

/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


| number of paramaters trunk: 15.229 M
| number of paramaters mask branch: 1.609 M
| number of paramaters score branch: 0.526 M
| number of paramaters total: 17.364 M
Session ind:  1 Obj ind:  1
Session ind:  1 Obj ind:  2
Session ind:  1 Obj ind:  3
Session ind:  1 Obj ind:  4
Session ind:  1 Obj ind:  5
Session ind:  1 Obj ind:  6
Session ind:  1 Obj ind:  7
Session ind:  1 Obj ind:  8
Session ind:  1 Obj ind:  9
Session ind:  1 Obj ind:  10
Session ind:  1 Obj ind:  11
Session ind:  1 Obj ind:  12
Session ind:  1 Obj ind:  13
Session ind:  1 Obj ind:  14
Session ind:  1 Obj ind:  15
Session ind:  1 Obj ind:  16
Session ind:  1 Obj ind:  17
Session ind:  1 Obj ind:  18
Session ind:  1 Obj ind:  19
Session ind:  1 Obj ind:  20
Session ind:  1 Obj ind:  21
Session ind:  1 Obj ind:  22
Session ind:  1 Obj ind:  23
Session ind:  1 Obj ind:  24
Session ind:  1 Obj ind:  25
Session ind:  1 Obj ind:  26
Session ind:  1 Obj ind:  27
Session ind:  1 Obj ind:  28
Session ind:  1 Obj ind:  29
S

In [ ]:
# root_dir_path = '/home/ehajizad/.avalanche/data/core50/core50_128x128_original/'
# save_dir_path = '/home/ehajizad/.avalanche/data/core50/core50_128x128/'
# ses_dirs = os.listdir(root_dir_path)
# for ses_dir in ses_dirs:
#     os.mkdir(os.path.join(save_dir_path,ses_dir))
#     obj_dirs = os.listdir(os.path.join(root_dir_path, ses_dir))
#     for obj_dir in obj_dirs:
#         os.mkdir(os.path.join(save_dir_path,ses_dir, obj_dir))
    